# 猫狗识别

- 数据预处理：图像数据处理，准备训练和验证数据集
- 卷积网络模型：构建网络架构
- 过拟合问题：观察训练和验证效果，针对过拟合问题提出解决方法
- 数据增强：图像数据增强方法与效果
- 迁移学习：深度学习必备训练策略

<img src="./img/1.png" alt="FAO" width="990">

In [ ]:
# import os
# from tensorflow.keras.preprocessing import image

# # 数据目录
# base_dir = "./data/cats_and_dogs"
# train_dir = os.path.join(base_dir, 'validation')

# # 创建目标增强数据目录
# output_base_dir = "./data/cats_and_dogs_2/validation"
# os.makedirs(os.path.join(output_base_dir, 'cats'), exist_ok=True)
# os.makedirs(os.path.join(output_base_dir, 'dogs'), exist_ok=True)

# # 创建数据增强生成器
# dataGen = image.ImageDataGenerator(
#     rescale=1./255,        # 归一化
#     rotation_range=40,     # 随机旋转角度
#     width_shift_range=0.2, # 随机水平移动
#     height_shift_range=0.2,# 随机垂直移动
#     shear_range=0.2,       # 随机剪切变换
#     zoom_range=0.2,        # 随机缩放
#     horizontal_flip=True,  # 随机水平翻转
#     fill_mode='nearest'    # 填充新创建像素的方法
# )

# # 为每个类别分别创建数据生成器
# for category in ['cats', 'dogs']:
#     in_path = os.path.join(train_dir, category)
#     out_path = os.path.join(output_base_dir, category)

#     # 创建数据生成器
#     gen_data = dataGen.flow_from_directory(
#         directory=train_dir,
#         classes=[category],  # 只生成当前类别的数据
#         batch_size=32,
#         shuffle=True,
#         save_to_dir=out_path,
#         save_prefix="gen",
#         target_size=(64, 64),
#         class_mode='binary'  # 设置为'binary' 或 'categorical'，根据你的分类需求
#     )

#     # 生成和保存图像
#     num_images = len(os.listdir(in_path))  # 获取原始图像数量
#     steps = num_images // 32  # 计算生成批次的次数
#     for _ in range(steps):  # 生成每一个批次
#         _ = next(gen_data)  # 使用 next() 获取下一个批次

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout

### 指定好数据路径（训练和验证）

In [ ]:
# 数据所在文件夹
base_dir = './data/cats_and_dogs_1'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# 训练集
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# 验证集
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

print("base_dir directory contents:", os.listdir(base_dir))
print("train_dir directory contents:", os.listdir(train_dir))
print("validation_dir directory contents:", os.listdir(validation_dir))
print("train_cats_dir directory contents:", os.listdir(train_cats_dir))
print("train_dogs_dir directory contents:", os.listdir(train_dogs_dir))
print("validation_cats_dir directory contents:", os.listdir(validation_cats_dir))
print("validation_dogs_dir directory contents:", os.listdir(validation_dogs_dir))

### 构建卷积神经网络模型
- 几层都可以，大家可以随意玩
- 如果用CPU训练，可以把输入设置的更小一些，一般输入大小更主要的决定了训练速度

In [ ]:
model = tf.keras.models.Sequential(
    [
        # 如果训练慢，可以把数据设置的更小一些
        # 获取32个特征图，卷积大小为3x3 激活函数为relu 输入大小为64x64x3（用于限制输入数据的尺寸，提醒作用）
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)),
        # 池化层，池化大小为2x2
        tf.keras.layers.MaxPooling2D(2, 2),
        Dropout(0.2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        Dropout(0.2),
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        # 为全连接层准备，转成1维矩阵数据
        tf.keras.layers.Flatten(),
        # 全连接层，神经元层个数设为512，激活函数为relu
        # 512个神经元，用于组合和处理从卷积层提取的特征。选择512个神经元是为了在特征处理能力和计算资源之间取得平衡。
        tf.keras.layers.Dense(512, activation="relu"),
        # 二分类sigmoid就够了，1个神经元，激活函数为sigmoid，用于二分类
        # sigmoid 表示在0和1之间，0表示狗，1表示猫
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [ ]:
model.summary()

配置训练器

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-4),
              metrics=['acc'])

### 数据预处理

- 读进来的数据会被自动转换成tensor(float32)格式，分别准备训练和验证
- 图像数据归一化（0-1）区间

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,  # 文件夹路径
    batch_size=20,
    target_size=(64, 64),
    # 如果one-hot就是categorical，二分类用binary就可以
    class_mode="binary",
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir, batch_size=20, class_mode="binary", target_size=(64, 64)
)

### 训练网络模型
- 直接fit也可以，但是通常咱们不能把所有数据全部放入内存，fit_generator相当于一个生成器，动态产生所需的batch数据
- steps_per_epoch相当给定一个停止条件，因为生成器会不断产生batch数据，说白了就是它不知道一个epoch里需要执行多少个step

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # 2000 images = batch_size * steps
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50,  # 1000 images = batch_size * steps
    verbose=2)

### 效果展示

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

看起来完全过拟合了，如何解决呢？

In [ ]:
# 保存模型
model.save("cat_dog_model.h5")